<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets

class RotationPredictionNet(nn.Module):
    def __init__(self):
        super(RotationPredictionNet, self).__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Linear(64 * 24 * 24, 4)

    def forward(self, x):
        x = self.convnet(x)
        return self.fc(x)

transform = transforms.Compose([
    transforms.RandomRotation((0, 360)),
    transforms.ToTensor()
])

dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

model = RotationPredictionNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    for images, _ in dataloader:
        images = torch.rot90(images, 1, [2, 3])
        labels = torch.tensor([1] * images.size(0))  # Assuming 90-degree rotation
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')